In [4]:
import pandas as pd
import os
import numpy as np
import warnings
from tqdm import tqdm
from functools import reduce

In [5]:
df1=pd.read_csv('EndZoneCFS_week1.csv', index_col=None)
df2=pd.read_csv('EndZoneCFS_week2.csv', index_col=None)
df3=pd.read_csv('EndZoneCFS_week3.csv', index_col=None)
df4=pd.read_csv('EndZoneCFS_week4.csv', index_col=None)
df5=pd.read_csv('EndZoneCFS_week5.csv', index_col=None)
df6=pd.read_csv('EndZoneCFS_week6.csv', index_col=None)
df7=pd.read_csv('EndZoneCFS_week7.csv', index_col=None)
df8=pd.read_csv('EndZoneCFS_week8.csv', index_col=None)
df9=pd.read_csv('EndZoneCFS_week9.csv', index_col=None)

df_counterfactuals=pd.concat([df1,df2, df3, df4,df5,df6, df7,df8, df9])


In [6]:
df_counterfactuals['Prediction Difference']=df_counterfactuals['Prediction Difference']*1.558
df_counterfactuals['Original Prediction']=df_counterfactuals['Original Prediction']*1.558
df_counterfactuals['Projected Prediction']=df_counterfactuals['Projected Prediction']*1.558

In [7]:
df_counterfactuals['PrevPrediction']=df_counterfactuals.groupby(['gameId', 'playId', 'nflId'])['Original Prediction'].transform(lambda x: x.shift(1))
df_counterfactuals['PrevYR']=df_counterfactuals.groupby(['gameId', 'playId', 'nflId'])['Yards Remaining'].transform(lambda x: x.shift(1))
df_counterfactuals['Prediction Difference-1']=df_counterfactuals.groupby(['gameId', 'playId', 'nflId'])['Prediction Difference'].transform(lambda x: x.shift(1))
df_counterfactuals['YardsSaved'] = df_counterfactuals['PrevPrediction'] - (df_counterfactuals['PrevYR'] - df_counterfactuals['Yards Remaining'] ) - df_counterfactuals['Original Prediction']
df_counterfactuals=df_counterfactuals.loc[(df_counterfactuals['YardsSaved'].isna()==False) & (df_counterfactuals['Prediction Difference-1'].isna()==False)]


In [8]:
frames = df_counterfactuals.groupby(['gameId', 'playId', 'frameId'])
framelist1 = []
#frames = df_counterfactuals[(df_counterfactuals['gameId'] == 2022092513) & (df_counterfactuals['playId'] == 320)].groupby(['gameId', 'playId', 'frameId'])
#frames = df_counterfactuals[(df_counterfactuals['gameId'] == 2022100205) & (df_counterfactuals['playId'] == 3163)].groupby(['gameId', 'playId', 'frameId'])
#frames = df_counterfactuals[(df_counterfactuals['gameId'] == 2022103004) & (df_counterfactuals['playId'] == 1570)].groupby(['gameId', 'playId', 'frameId'])
for frame in tqdm(frames):

    frame[1].loc[frame[1]['Prediction Difference-1'] <= 0,'Prediction Difference-1'] = 0  
    if sum(frame[1].loc[:,'Prediction Difference-1']) == 0:
        frame[1].loc[:,'Prediction Difference-1'] = 1
    x = (frame[1]['YardsSaved'].iloc[0])/sum((frame[1]['Prediction Difference-1']))
    frame[1]['NormalizedYSAX'] = frame[1]['Prediction Difference-1']*x
    framelist1 += [frame[1]]
    
'''players = pd.read_csv(os.path.join(os.path.join("Data", str(2024)),"players.csv"))
frame = pd.concat(framelist1, ignore_index=True)
print(sum(frame['YardsSaved'])/11)
frame=pd.DataFrame({'NormalizedYSAX':frame.groupby(['nflId'])['NormalizedYSAX'].sum()}).sort_values(by='NormalizedYSAX', ascending=False).reset_index()
frame = frame.merge(players[['nflId','displayName','position']], how='left', on=['nflId'])
frame,  sum(frame['NormalizedYSAX'])'''

100%|██████████| 300923/300923 [02:50<00:00, 1769.94it/s]


'players = pd.read_csv(os.path.join(os.path.join("Data", str(2024)),"players.csv"))\nframe = pd.concat(framelist1, ignore_index=True)\nprint(sum(frame[\'YardsSaved\'])/11)\nframe=pd.DataFrame({\'NormalizedYSAX\':frame.groupby([\'nflId\'])[\'NormalizedYSAX\'].sum()}).sort_values(by=\'NormalizedYSAX\', ascending=False).reset_index()\nframe = frame.merge(players[[\'nflId\',\'displayName\',\'position\']], how=\'left\', on=[\'nflId\'])\nframe,  sum(frame[\'NormalizedYSAX\'])'

In [9]:
results1 = pd.concat(framelist1, ignore_index=True)

In [10]:
results1.to_csv('finalrankings.csv') 


In [11]:
import warnings
warnings.filterwarnings('ignore')


In [12]:
results = pd.read_csv('finalrankings.csv')
#print(np.mean(results[results['NormalizedYSAX'] < 0]['Normalized YSAX']))
results['NormalizedYSAX'] = results['NormalizedYSAX'].fillna(0)
players = pd.read_csv(os.path.join(os.path.join("Data", str(2024)),"players.csv"))

tackles = pd.read_csv(os.path.join(os.path.join("Data", str(2024)),"tackles.csv"))
tacklesum = pd.DataFrame(tackles.groupby(['nflId'])['tackle'].sum()).sort_values(by='tackle', ascending=False).reset_index()
assistsum = pd.DataFrame(tackles.groupby(['nflId'])['assist'].sum()).sort_values(by='assist', ascending=False).reset_index()
missedsum = pd.DataFrame(tackles.groupby(['nflId'])['pff_missedTackle'].sum()).sort_values(by='pff_missedTackle', ascending=False).reset_index()
totaltackles = [tacklesum,assistsum,missedsum]
totaltackles = reduce(lambda left, right: pd.merge(left,right, on = ['nflId'], how = 'outer'),totaltackles).fillna(0)
totaltackles['TackleValue'] = totaltackles['tackle'] + 0.5*totaltackles['assist'] - totaltackles['pff_missedTackle']

results =pd.DataFrame(results.groupby(['nflId','playId'])['NormalizedYSAX'].sum()).sort_values(by='NormalizedYSAX', ascending=False).reset_index()
results = results.merge(players, how='left', on=['nflId'])
results['SnapCount'] = results.groupby(['nflId'])['nflId'].transform('count')
results=pd.DataFrame({'NormalizedYSAX': results.groupby(['nflId'])['NormalizedYSAX'].sum(), 'SnapCount':results.groupby(['nflId'])['SnapCount'].sum()}).sort_values(by='NormalizedYSAX', ascending=False).reset_index()
results['SnapCount'] = results['SnapCount']**0.5
results = results.merge(players, how='left', on=['nflId'])
results = results[['NormalizedYSAX','position','displayName','nflId','SnapCount']]

players = players[players['position'].isin(['CB', 'DB', 'DE', 'DT', 'FS', 'ILB', 'MLB', 'NT', 'OLB', 'SS'])]
rankings = players.merge(totaltackles[['nflId','TackleValue']], how='left', on=['nflId']).fillna(0)
rankings = rankings.merge(results[['nflId','NormalizedYSAX','SnapCount']], how='left', on=['nflId']).fillna(0)
rankings = rankings[rankings['SnapCount'] > 0]
rankings['TackleValue/Snap'] = rankings['TackleValue']/rankings['SnapCount']
rankings['NormalizedYSAX/Snap'] = rankings['NormalizedYSAX']/rankings['SnapCount']
rankings = rankings[['displayName','nflId','position','SnapCount','NormalizedYSAX','NormalizedYSAX/Snap','TackleValue','TackleValue/Snap']]
rankings = rankings.sort_values(by='NormalizedYSAX', ascending=False)


In [16]:
LB = rankings.loc[(rankings['position']=='ILB') | (rankings['position']=='OLB') | (rankings['position']=='MLB')]
LB.head(10).to_csv('LBrankings.csv')

In [17]:
S = rankings.loc[(rankings['position']=='FS') | (rankings['position']=='SS') | (rankings['position']=='CB')]
S.head(10).to_csv('Srankings.csv')

In [18]:
DE = rankings.loc[(rankings['position']=='DT') | (rankings['position']=='NT') | (rankings['position']=='DE')].head(10)
DE.head(10).to_csv('DErankings.csv')